In [3]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

INPUT_PATH = "/content/datafile.tsv"
OUTPUT_DIR = "/content/"

CATS = [1, 2, 3, 4, 5, 6, 7]

# Helper Functions
def normalize_columns(df):
    """Normalize column names and types"""
    df.columns = [c.strip() for c in df.columns]

    col_map = {
        "Provider": "provider", "provider": "provider",
        "Model": "model", "name": "model",
        "Category": "category", "category": "category",
        "Score": "score", "score": "score",
        "Country": "country", "country": "country",
        "Sector": "sector", "sector": "sector",
        "Year": "year", "year": "year",
        "Metadata": "metadata", "metadata": "metadata"
    }
    df = df.rename(columns=col_map)
    df["category"] = pd.to_numeric(df["category"], errors="coerce").astype("Int64")
    df["score"] = pd.to_numeric(df["score"], errors="coerce")
    df["provider"] = df["provider"].astype(str).str.strip()

    # Add missing columns with defaults
    if "country" not in df.columns:
        df["country"] = "Unknown"
    else:
        df["country"] = df["country"].astype(str).str.strip().replace({"": "Unknown"})

    if "sector" not in df.columns:
        df["sector"] = "Unknown"
    else:
        df["sector"] = df["sector"].astype(str).str.strip().replace({"": "Unknown"})

    if "year" not in df.columns:
        df["year"] = pd.Series([np.nan]*len(df), dtype="Int64")

    return df


def filter_model_releases(df):
    """Filter to model releases only"""
    if "is_model_release" not in df.columns:
        return df

    df["is_model_release"] = df["is_model_release"].map({
        True: True, False: False,
        "True": True, "False": False,
        "true": True, "false": False,
        1: True, 0: False
    })

    return df[df["is_model_release"] == True].copy()


def extract_release_date(val):
    """Extract date from metadata string"""
    if pd.isna(val):
        return None
    s = str(val)
    m = re.search(r"\b(20\d{2})-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])\b", s)
    if not m:
        return None
    try:
        return datetime.strptime(m.group(0), "%Y-%m-%d").date()
    except Exception:
        return None


def add_temporal_features(df):
    """Add year and quarter columns from metadata"""
    df["release_date_parsed"] = df.get("metadata", pd.Series([np.nan]*len(df))).apply(extract_release_date)

    # Fill year from parsed date if missing
    df.loc[df["year"].isna() & df["release_date_parsed"].notna(), "year"] = \
        pd.to_numeric(df.loc[df["release_date_parsed"].notna(), "release_date_parsed"].apply(lambda d: d.year), errors="coerce").astype("Int64")

    # Create quarter labels
    def to_quarter_label(d):
        if pd.isna(d):
            return "Unknown"
        month = d.month
        q = f"Q{(month-1)//3 + 1}"
        return f"{d.year}-{q}"

    df["release_quarter"] = df["release_date_parsed"].apply(lambda d: to_quarter_label(d) if pd.notna(d) else "Unknown")

    return df


def create_model_matrix(df, group_cols):
    """Create per-model category matrix with missing categories filled as 0"""
    # Average duplicates at model level
    df_model_cat = df.groupby(group_cols + ["category"], as_index=False)["score"].mean()

    # Pivot and fill missing categories with 0
    mat_model = (
        df_model_cat
        .pivot_table(index=group_cols, columns="category", values="score", aggfunc="mean")
        .reindex(columns=CATS)
        .fillna(0.0)
    )

    return mat_model


def aggregate_and_save(mat_model, group_level, output_name):
    """Aggregate model matrix by group level and save"""
    mat_agg = mat_model.groupby(level=group_level).mean()
    mat_agg["overall_avg"] = mat_agg[CATS].mean(axis=1)
    mat_agg = mat_agg.round(2)

    output_path = f"{OUTPUT_DIR}{output_name}"
    mat_agg.reset_index().rename_axis(None, axis=1).to_csv(output_path, sep="\t", index=False)
    print(f"Saved {group_level} averages to: {output_path}")

    return mat_agg

In [4]:
def main():
    df = pd.read_csv(INPUT_PATH, sep="\t")
    df = normalize_columns(df)
    df = filter_model_releases(df)
    df = add_temporal_features(df)

    # Keep only valid categories
    df = df[df["category"].between(1, 7, inclusive="both")]

    # Check required columns
    required = {"provider", "model", "category", "score"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {sorted(missing)}")

    print(f"Filtered to {len(df)} model release rows")

    # Create per-model matrix
    print("\nCreating model-level category matrix...")
    group_cols = ["country", "sector", "provider", "model", "year", "release_quarter"]
    mat_model = create_model_matrix(df, group_cols)
    mat_model = mat_model.round(2)

    # Save model matrix
    model_out = f"{OUTPUT_DIR}model_category_scores_matrix.tsv"
    mat_model.reset_index().rename_axis(None, axis=1).to_csv(model_out, sep="\t", index=False)
    print(f"Saved per-model matrix to: {model_out}")

    # Aggregate by social impact dimensions
    mat_provider = aggregate_and_save(mat_model, "provider", "provider_category_avg_scores.tsv")
    mat_country = aggregate_and_save(mat_model, "country", "country_category_avg_scores.tsv")
    mat_sector = aggregate_and_save(mat_model, "sector", "sector_category_avg_scores.tsv")
    mat_year = aggregate_and_save(mat_model, "year", "year_category_avg_scores.tsv")
    mat_quarter = aggregate_and_save(mat_model, "release_quarter", "quarter_category_avg_scores.tsv")

if __name__ == "__main__":
    main()

Filtered to 1197 model release rows

Creating model-level category matrix...
Saved per-model matrix to: /content/model_category_scores_matrix.tsv
Saved provider averages to: /content/provider_category_avg_scores.tsv
Saved country averages to: /content/country_category_avg_scores.tsv
Saved sector averages to: /content/sector_category_avg_scores.tsv
Saved year averages to: /content/year_category_avg_scores.tsv
Saved release_quarter averages to: /content/quarter_category_avg_scores.tsv
